# Coordle Idea
1. Get query, e.g. "What is love?"
2. Tokenize query
3. Create a vector using word2vec of the tokens by summing them, or finding the average or whatever.
   lets call it the query vector. 
4. Find sentence vectors that are the closesest to the query vector.
5. Return the papers corresponding to said sentence vectors.

# Coordle Backend

In [79]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
import en_core_sci_lg # Biomedical word embeddings
from utils import clean_text, load_pickle, save_pickle
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque
from copy import deepcopy
from collections.abc import Iterable
from typing import Union
from pprint import pprint
from itertools import chain
import pymongo


import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

In [83]:
import inspect
import sys

print(__name__, sys.argv[0])

__main__ /home/jonavin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py


In [2]:
df = pd.read_csv('data/cord-19-data.csv')

In [3]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''
class DocEpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

In [4]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec', 'w2v_model_epoch_29.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [5]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import (CordDoc, Index, RecursiveDescentParser, 
                             AI_Index)

ai_index = AI_Index(model.wv.most_similar, n_similars=1)
ai_index.build_from_df(
    df[:16],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 16/16 [00:00<00:00, 280.75it/s]


In [6]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index, RecursiveDescentParser, AI_Index

docs, scores, errmsgs = ai_index.search('virus')

n = 69
if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}  {str(doc.title)[:70]:<70}  {score:.4f}')


1wswi7us  Relationship of SARS-CoV to other pathogenic RNA viruses explored by t  39.6302
yy96yeu9  Viral Discovery and Sequence Recovery Using DNA Microarrays             15.6958
xqhn0vbp  Airborne rhinovirus detection and effect of ultraviolet irradiation on  10.8963
qj4dh6rg  Cloaked similarity between HIV-1 and SARS-CoV suggests an anti-SARS st  8.0116
5s6acr7m  The Virus That Changed My World                                         7.2854
le0ogx1s  A new recruit for the army of the men of death                          7.0304
0qaoam29  A double epidemic model for the SARS propagation                        5.1278
gi6uaa83  Discovering human history from stomach bacteria                         2.6177
ng4rrdte  Pro/con clinical debate: Steroids are a key component in the treatment  1.9475
fy4w7xz8  Association of HLA class I with severe acute respiratory syndrome coro  0.9052
1769ovyk  8th Annual Toronto Critical Care Medicine Symposium, 30 October–1 Nove  0.4316
kuybfc1y  Descript

In [7]:
ai_index.docmap['grace']

{1769ovyk}

In [8]:
with open('textfile.txt', 'w+') as f:
    uid = 'qzm9wgde'
    
    print(f'dumping {uid}')
    f.write(df[df.cord_uid == uid].body_text.values[0].lower())

dumping qzm9wgde


# Mongo

In [62]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import (CordDoc, Index, RecursiveDescentParser, 
                             AI_Index)

ram_index = Index()
# ram_index = AI_Index(model.wv.most_similar, 1)
ram_index.build_from_df(
    df[:2048],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 2048/2048 [00:07<00:00, 265.99it/s]


In [76]:
import coordle_mongobackend as cm
reload(cm)

# mongoindex = cm.Index('coordle')
mongoindex = cm.AI_Index('coordle', model.wv.most_similar, 3)
# mongoindex.drop_old_collections()
# mongoindex.build_from_df(df[:2048], 'cord_uid', 'title', 'body_text', 
#                          use_multiprocessing=True, workers=-1, verbose=True)

Initialized database index with size 2048


In [75]:
docs, scores, errmsgs = ram_index.search('dangerous virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}   {score:.4f}')

deque(['dangerous', 'OR', 'virus'])
yz2lku1i   0.5327
okdiyk1m   0.5311
x4hdxx1t   0.4183
9sso5gfj   0.3755
bhq8t5nq   0.3669
ykhhnsd3   0.3571
kc8hw8ql   0.3492
36gbl7tu   0.3436
b378ppd8   0.3421
ujtb7xgz   0.3410
f0vud3gu   0.3366
1wswi7us   0.3248
bptfnaph   0.3239
629kl04a   0.3226
mbtifn3p   0.3188
z8zkl2xh   0.3181
kxwgpymq   0.3106
zq387qo8   0.3097
5blxgsfi   0.3056
pqi6wjjo   0.3051
u49t8na9   0.3016
k6v4am7l   0.2976
qz68gtd4   0.2937
tg8jdol5   0.2899
i55gwn3s   0.2816
7gmtp6km   0.2755
dafpoz3r   0.2666
t98ed1gb   0.2661
87yq2317   0.2605
w1nxt8iy   0.2568
0ewu7e1c   0.2556
v4op73hf   0.2549
5kiu2pb6   0.2542
gf718lbl   0.2529
4txctk7k   0.2527
xz7rzjqv   0.2473
6u6ff1d7   0.2464
t3etupgh   0.2461
d2jbuhct   0.2461
w2efsshj   0.2459
3v4vkcit   0.2421
qos9vu3r   0.2385
73vsu07m   0.2372
x3b6j5d0   0.2345
y77c4ro4   0.2338
m57b8g9n   0.2314
38kg24nv   0.2298
g1rxu6xo   0.2298
gvfz6qio   0.2298
vg8bwed9   0.2287
5y9kfyk9   0.2242
v95fzp8n   0.2224
iuf0kd20   0.2218
96w28ygp  

In [77]:
docs, scores, errmsgs = mongoindex.search('dangerous virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc}   {score:.4f}')

deque(['dangerous', 'OR', 'threatening', 'OR', 'deadly', 'OR', 'hazardous', 'OR', 'virus', 'OR', 'viruses', 'OR', 'viral', 'OR', 'coronavirus'])
ujhgb3b0   0.3987
602a2xdn   0.3681
k6v4am7l   0.3209
xz7rzjqv   0.3050
yy96yeu9   0.2830
mbtifn3p   0.2819
ygnbt4jb   0.2791
1wswi7us   0.2749
ffk58hoo   0.2706
okdiyk1m   0.2667
lal35iyr   0.2613
bptfnaph   0.2593
fuzvu7y1   0.2550
477utyfm   0.2537
yz2lku1i   0.2520
q5msn97r   0.2492
pqi6wjjo   0.2412
t7txxn99   0.2406
0lbxvudt   0.2399
z8zkl2xh   0.2397
x3b6j5d0   0.2349
x4hdxx1t   0.2283
ykhhnsd3   0.2252
96w28ygp   0.2251
sbnnh2mm   0.2223
5gsbtfag   0.2222
iu3dg7y1   0.2201
q41elfwt   0.2134
zq387qo8   0.2129
fdg72g0f   0.2126
629kl04a   0.2118
awp2koqb   0.2116
ujtb7xgz   0.2106
9a56rr1j   0.2100
ki9f6ssk   0.2069
36gbl7tu   0.2065
sprvon1t   0.2059
jh9e85c0   0.2025
iwearl9q   0.2007
0fitbwuv   0.1976
qdgqif5j   0.1953
9b6yz3f4   0.1924
dwjijmwz   0.1923
3jaw7ymu   0.1915
le0ogx1s   0.1914
3ahp9tli   0.1911
7bkf7wrs   0.1901
rxbcuseq 

In [13]:
wordmap = {word:[doc.uid for doc in docs] for word, docs in ai_index.docmap.items() if len(word) < 100}

In [14]:
A = pd.value_counts(['a','a','a','d','c','c','c','c','d'], sort=False)
A.index.values

array(['a', 'c', 'd'], dtype=object)

In [15]:
A = [[],0]
A[0].append(1)
A[1] += 1
A

[[1], 1]

In [16]:
client = pymongo.MongoClient()
db_test = client['coordle']
db_test.wordcounts.count_documents({})

2048

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<center><h2> Junkyard </h2></center>

In [17]:
raise ValueError

ValueError: 

Creating sets from dicts is faster than from lists

In [ ]:
from time import time 
from matplotlib import pyplot as plt 
import numpy as np

dict_time = []
list_time = []
ratios = []

xrange = np.arange(1000,1000000,1000)
B = set(list(range(100000)))

for j in xrange:
    a = list(range(j))

    t0=time()
    A = set(a)
    A | B
    t1_list = time()-t0
    list_time.append(t1_list)
    
    a = {i:None for i in range(j)}
    
    t0=time()
#     A = set(a)
    a.keys() | B
    t1_dict = time()-t0
    dict_time.append(t1_dict)

    ratios.append(t1_list/t1_dict)

In [ ]:
plt.plot(xrange[1:], dict_time[1:], label='dict')
plt.plot(xrange[1:], list_time[1:], label='list')
plt.legend()
plt.show()

In [ ]:
plt.plot(xrange[1:], ratios[1:])
plt.show()

In [ ]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index2, RecursiveDescentParser, AI_Index2
Index = Index2

queries = [
    'retarded!!!',
#     '(',
#     ')',
    'retarded (white AND (woman NOT man))',
    'retarded (white AND white) man',
#     'retarded OR white OR woman',
#     'retarded white AND woman',
#     'retarded OR white NOT woman',
#     'retarded (white NOT woman)',
#     'retarded (white NOT woman)',
#     'OR retarded AND white woman',
#     'retarded AND AND white NOT woman',
#     'retarded (white NOT woman) AND',
#     ')retarded ((white NOT woman) AND',
#     'retarded ((white NOT woman)',
#     'AND retarded)) ((white NOT woman) NOT',
]

rdp = RecursiveDescentParser(index.docmap)
for query in queries:
    errmsgs = []
    tokens = rdp.get_logical_querytokens(query)
    pass_ = rdp.assert_query(tokens, errmsgs)
    
#     print(tokens)
    print(rdp.parenthesis_handler(tokens))
#     print(errmsgs)
    print(pass_)
    print()
    